In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "2001"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 2.2 count all captures
t1.count()

47162716

In [5]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [6]:
df.show()

+--------------------+--------------------+------+---------+
|         originalUrl|              digest|status|     mime|
+--------------------+--------------------+------+---------+
|http://www.stopkl...|CFJY5WTXRWLZM3B5Q...|   301|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|P4C4JU3ZFHYQ6OP46...|   301|text/html|
|http://www.stopkl...|UFY6TJRXESSK4SW5M...|   301|text/html|
|http://www.stopkl...|UFY6TJRXESSK4SW5M...|   301|text/html|
|http://www.stopkl...|RHSJSKHLN5PUQBUNK...|   301|text/html|
|http://www.stopkl...|RHSJSKHLN5PUQBUNK...|   301|text/html|
|http://www.stopkl...|5J7D5UTD5RRF2HA6N...|   301|text/html|
|http://www.stopkl...|DSQPHWBGC2GSCTAFH...|   301|text/html|
|http://www.stopkl...|MVMN5J4NQUDHP65SZ...|   301|text/html|
|http://www.stopkl...|MVMN5J4NQUDHP65SZ...|   301|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|DS

In [7]:
// 4. Count distinct originalUrls
val dOu = df.select(df("originalUrl")).distinct.count()

In [8]:
dOu

13918167

In [9]:
// 5. srednia i mediana liczby wersji na pojedynczy URL
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._

val mDu = df.groupBy("originalUrl").agg(count("digest") as "digests").orderBy(desc("digests")).cache()

In [10]:
mDu.show(false)

+--------------------------------------+-------+
|originalUrl                           |digests|
+--------------------------------------+-------+
|http://www.wp.pl:80/robots.txt        |35116  |
|http://of.pl:80/robots.txt            |27586  |
|http://www.hg.pl:80/robots.txt        |25698  |
|http://tools.ptc.pl:80/robots.txt     |15436  |
|http://www.hg.pl:80/404.html          |15152  |
|http://www.multinet.pl:80/robots.txt  |13486  |
|http://www.alpha.pl:80/robots.txt     |12768  |
|http://www.net-line.pl:80/robots.txt  |12134  |
|http://www.itt.pl:80/robots.txt       |11462  |
|http://www.alpha.pl:80/missing.html   |9500   |
|http://katalog.wp.pl:80/www/index.html|8606   |
|http://www.atcom.net.pl:80/robots.txt |8422   |
|http://www.kki.net.pl:80/             |8364   |
|http://free.eroskop.pl:80/robots.txt  |8234   |
|http://viper.pl:80/robots.txt         |8116   |
|http://error.punkt.pl:80/robots.txt   |8096   |
|http://www.netservice.pl:80/robots.txt|8038   |
|http://www.toya.net

In [11]:
// 5.1 mean
mDu.agg(avg("digests")).show()

+----------------+
|    avg(digests)|
+----------------+
|3.38857236013909|
+----------------+



In [12]:
// 5.2 median
mDu.agg(expr("percentile(digests, 0.5)").as("median")).cache().show()

+------+
|median|
+------+
|   2.0|
+------+



In [13]:
// tab 2. A - statusy inne niz 200
df.where("status != '200'").count()

10818288

In [14]:
// tab 2. A - statusy HTML inne niz 200
df.where("mime == 'text/html'").where("status != '200'").count()

10767076

In [15]:
// test pls
df.where("status != '200'").show(150)

+--------------------+--------------------+------+---------+
|         originalUrl|              digest|status|     mime|
+--------------------+--------------------+------+---------+
|http://www.stopkl...|CFJY5WTXRWLZM3B5Q...|   301|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|P4C4JU3ZFHYQ6OP46...|   301|text/html|
|http://www.stopkl...|UFY6TJRXESSK4SW5M...|   301|text/html|
|http://www.stopkl...|UFY6TJRXESSK4SW5M...|   301|text/html|
|http://www.stopkl...|RHSJSKHLN5PUQBUNK...|   301|text/html|
|http://www.stopkl...|RHSJSKHLN5PUQBUNK...|   301|text/html|
|http://www.stopkl...|5J7D5UTD5RRF2HA6N...|   301|text/html|
|http://www.stopkl...|DSQPHWBGC2GSCTAFH...|   301|text/html|
|http://www.stopkl...|MVMN5J4NQUDHP65SZ...|   301|text/html|
|http://www.stopkl...|MVMN5J4NQUDHP65SZ...|   301|text/html|
|http://www.stopkl...|KYTLZGHZ6ENI2FE7Q...|   404|text/html|
|http://www.stopkl...|DS

In [16]:
// tab basic2.2
// count distinct mime http 200
df.where("status == '200'").agg(countDistinct("mime")).show()

+--------------------+
|count(DISTINCT mime)|
+--------------------+
|                 248|
+--------------------+



In [17]:
// count all mimes counts
df.where("status == '200'").groupBy("mime").count().orderBy(desc("count")).cache().show(250)

+--------------------+--------+
|                mime|   count|
+--------------------+--------+
|           text/html|27328568|
|           image/gif| 3449178|
|          image/jpeg| 2896882|
|           alexa/dat| 1384634|
|          text/plain|  849752|
|            text/css|  115564|
|application/octet...|   87790|
|           image/png|   82098|
|application/x-jav...|   55150|
|                 unk|   12556|
|     application/zip|   10212|
|            text/xml|    8188|
|audio/x-pn-realaudio|    7736|
|application/x-msd...|    5064|
|     application/pdf|    4578|
|application/corel...|    4208|
|           image/bmp|    3706|
|            text/vnd|    2926|
|          audio/mpeg|    2792|
|     audio/x-mpegurl|    2562|
|   application/x-tar|    2244|
|application/posts...|    2118|
|         image/pjpeg|    1954|
|application/x-sho...|    1864|
|  application/msword|    1548|
|     image/x-xbitmap|    1442|
|         audio/x-wav|    1318|
|   application/x-tex|    1040|
|       

In [18]:
df.where("status == '200'").groupBy("mime").count().orderBy(desc("mime")).cache().show(250)

+--------------------+--------+
|                mime|   count|
+--------------------+--------+
|      x-world/x-vrml|     166|
|         www/unknown|     144|
|                 wml|       6|
|            vms/help|       6|
|             vms/exe|      32|
|video/x-pn-realvi...|       8|
|     video/x-msvideo|     216|
|      video/x-ms-wmv|      30|
|      video/x-ms-asf|     274|
|         video/x-fli|       2|
|       video/unknown|      48|
|     video/quicktime|     218|
|          video/mpeg|     680|
|           video/avi|       2|
|                 unk|   12556|
|        udio/x-scpls|      48|
|             tks/vnd|       4|
|            text/xml|    8188|
|        text/x-vcard|      32|
|          text/x-tex|     106|
|          text/x-tcl|     154|
|           text/x-sh|      36|
|         text/x-sgml|      20|
|       text/x-setext|       8|
|text/x-server-par...|     142|
|       text/x-pascal|      98|
|   text/x-javascript|       6|
|         text/x-java|     208|
| text/x

In [31]:
// for tests - liczba wersji na origina;Url

val vBu = df.groupBy("originalUrl").agg(countDistinct("digest") as "distDigests").cache()

In [32]:
vBu.show()

+--------------------+-----------+
|         originalUrl|distDigests|
+--------------------+-----------+
|http://www.sunsmi...|          3|
|http://www.tai.co...|          1|
|http://www.tchik....|          1|
|http://www.techla...|          3|
|http://unikom.com...|          2|
|http://www.win.co...|          1|
|http://www.wirtua...|          2|
|http://www.wirusy...|          1|
|http://spider.wmc...|          3|
|http://www.wsip.c...|          1|
|http://www.wyborc...|          1|
|http://www.zoltri...|          1|
|http://www.hotele...|          1|
|http://katalog.wp...|          2|
|http://kuchnia.wp...|          2|
|http://szukaj.wp....|          1|
|http://szukaj.wp....|          1|
|http://wyborcza.g...|         48|
|http://www.muzeum...|          1|
|http://cenzor.zie...|          1|
+--------------------+-----------+
only showing top 20 rows



In [33]:
// avg
vBu.agg(avg("distDigests") as "avgDigests").show()

+------------------+
|        avgDigests|
+------------------+
|1.1655159763494718|
+------------------+



In [34]:
vBu.agg(expr("percentile(distDigests, 0.5)").as("medDigests")).cache().show()

+----------+
|medDigests|
+----------+
|       1.0|
+----------+

